# Q2. Download and preprocess the data

In [1]:
## Create Folder
# !mkdir data
# %cd data
## January
# !curl https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet --output green_tripdata_2023-01.parquet
## February
# !curl https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet --output green_tripdata_2023-02.parquet
## March
# !curl https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet --output green_tripdata_2023-03.parquet

# %cd ..

In [2]:
!python ./src/preprocess_data.py --raw_data_path ./data --dest_path ./output

# Q3. Train a model with autolog

In [3]:
!python ./src/train.py --data_path ./output

2024/05/30 01:25:35 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/05/30 01:25:36 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

In [4]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///./db/mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
latest_run = client.search_runs(experiment_ids="1", order_by=["attributes.start_time desc"], max_results=1)
print(
    f"min_samples_split: {latest_run[0].data.params['min_samples_split']}")

min_samples_split: 2


# Q4. Launch the tracking server locally

In [6]:
# !mlflow ui --backend-store-uri sqlite:///./db/mlflow.db --default-artifact-root ./mlruns

# Q5. Tune model hyperparameters

In [7]:
# !python ./src/hpo.py

![UI Screenshot](./assets/random_forest_hyperopt.png)

# Q6. Promote the best model to the model registry

In [11]:
!python ./src/register_model.py

2024/05/30 02:17:26 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/30 02:17:40 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/amiralism/miniconda3/envs/bio/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2024/05/30 02:19:07 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/30 02:19:39 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/30 02:19:48 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/30 02:19:59 WARNING mlflow.sklearn: Failed to log training dataset information to M